In [2]:
D1 = "vaccine research corona virus research"
D2 = "research research cancer vaccine vaccine"
D3 = "virus virus corona vaccine lab"
D4 = "cancer lab research lab"

In [3]:
corpus = [D1,D2,D3,D4]
corpus

['vaccine research corona virus research',
 'research research cancer vaccine vaccine',
 'virus virus corona vaccine lab',
 'cancer lab research lab']

In [4]:
wordbags = set(D1.split() + D2.split() + D3.split() + D4.split() )
wordbags

{'cancer', 'corona', 'lab', 'research', 'vaccine', 'virus'}

In [5]:
total_words = len(D1.split() + D2.split() + D3.split() + D4.split())
total_words

19

In [6]:
import pandas as pd

P_Mc_t = {}

for wb in wordbags:
  for d in corpus:
    cnt = d.count(wb)
    
    if wb in P_Mc_t:
      P_Mc_t[wb] += cnt
    else:
      P_Mc_t[wb] = cnt
  
P_Mc_t 

{'corona': 2, 'cancer': 2, 'lab': 3, 'virus': 3, 'research': 5, 'vaccine': 4}

In [7]:
P_Mc_t = pd.DataFrame(P_Mc_t,index=['word_count']).transpose()
P_Mc_t['P_Mc(t)'] = P_Mc_t['word_count'] / total_words
P_Mc_t

,word_count,P_Mc(t)
corona,2,0.105263
cancer,2,0.105263
lab,3,0.157895
virus,3,0.157895
research,5,0.263158
vaccine,4,0.210526


In [8]:
P_Mc_t['prob_wc_c'] = P_Mc_t['word_count'] / total_words

In [9]:
PMd_t = {}

for wb in wordbags:
    PMd = []
    for d in corpus:
      cnt = d.count(wb)
      PMd.append(cnt/len(d.split()))
    
    PMd_t[wb] = PMd   
  
PMd_t 

{'corona': [0.2, 0.0, 0.2, 0.0],
 'cancer': [0.0, 0.2, 0.0, 0.25],
 'lab': [0.0, 0.0, 0.2, 0.5],
 'virus': [0.2, 0.0, 0.4, 0.0],
 'research': [0.4, 0.4, 0.0, 0.25],
 'vaccine': [0.2, 0.4, 0.2, 0.0]}

In [10]:
df = pd.DataFrame(PMd_t,index=['d1','d2','d3','d4']).transpose()
df.sort_index()

,d1,d2,d3,d4
cancer,0.0,0.2,0.0,0.25
corona,0.2,0.0,0.2,0.00
lab,0.0,0.0,0.2,0.50
research,0.4,0.4,0.0,0.25
vaccine,0.2,0.4,0.2,0.00
virus,0.2,0.0,0.4,0.00


In [11]:
query = "vaccine vaccine research cancer"

data={}

for i in df.columns:
  data[i] = 1
  for w in query.split():
    data[i] = df[i][w] * data[i]

data

{'d1': 0.0, 'd2': 0.012800000000000004, 'd3': 0.0, 'd4': 0.0}

In [12]:
query = "vaccine vaccine research cancer"
lamda = 0.5

data={}

for i in df.columns:
  data[i] = 1
  for w in query.split():

    data[i] = ((lamda * df[i][w]) + ((1-lamda) * P_Mc_t['P_Mc(t)'][w])) * data[i]

data 

{'d1': 0.0007352844131030303,
 'd2': 0.004716068784002579,
 'd3': 0.000291779529009139,
 'd4': 0.0005050030309773557}

In [13]:
import operator
sorted(data.items(), key=operator.itemgetter(1),reverse=True)

[('d2', 0.004716068784002579),
 ('d1', 0.0007352844131030303),
 ('d4', 0.0005050030309773557),
 ('d3', 0.000291779529009139)]

In [14]:
query = "vaccine research"
lamda = 0.5

data={}

for i in df.columns:
  data[i] = 1
  for w in query.split():

    data[i] = ((lamda * df[i][w]) + ((1-lamda) * P_Mc_t['P_Mc(t)'][w])) * data[i]

data 

{'d1': 0.06806094182825485,
 'd2': 0.10121883656509696,
 'd3': 0.027008310249307478,
 'd4': 0.027008310249307475}

In [15]:
import operator
sorted(data.items(), key=operator.itemgetter(1),reverse=True)

[('d2', 0.10121883656509696),
 ('d1', 0.06806094182825485),
 ('d3', 0.027008310249307478),
 ('d4', 0.027008310249307475)]

In [16]:
def readData():
    data = ['the weather forecast in Koblenz','next week weather forecast','the weather tomorrow','weather forecast Amsterdam',' the weather tomorrow in']
    dat=[]
    for i in range(len(data)):
        for word in data[i].split():
            dat.append(word)
    print(dat)
    return dat

def createBigram(data):
   listOfBigrams = []
   bigramCounts = {}
   unigramCounts = {}
   for i in range(len(data)-1):
      if i < len(data) - 1 and data[i+1].islower():

         listOfBigrams.append((data[i], data[i + 1]))

         if (data[i], data[i+1]) in bigramCounts:
            bigramCounts[(data[i], data[i + 1])] += 1
         else:
            bigramCounts[(data[i], data[i + 1])] = 1

      if data[i] in unigramCounts:
         unigramCounts[data[i]] += 1
      else:
         unigramCounts[data[i]] = 1
   return listOfBigrams, unigramCounts, bigramCounts


def calcBigramProb(listOfBigrams, unigramCounts, bigramCounts):
    listOfProb = {}
    for bigram in listOfBigrams:
        word1 = bigram[0]
        word2 = bigram[1]
        listOfProb[bigram] = (bigramCounts.get(bigram))/(unigramCounts.get(word1))
    return listOfProb


if __name__ == '__main__':
    data = readData()
    listOfBigrams, unigramCounts, bigramCounts = createBigram(data)

    print("\n All the possible Bigrams are ")
    print(listOfBigrams)

    print("\n Bigrams along with their frequency ")
    print(bigramCounts)

    print("\n Unigrams along with their frequency ")
    print(unigramCounts)

    bigramProb = calcBigramProb(listOfBigrams, unigramCounts, bigramCounts)

    print("\n Bigrams along with their probability ")
    print(bigramProb)
    inputList="the weather"
    splt=inputList.split()
    outputProb1 = 1
    bilist=[]
    bigrm=[]

    for i in range(len(splt) - 1):
        if i < len(splt) - 1:

            bilist.append((splt[i], splt[i + 1]))

    print("\n The bigrams in given sentence are ")
    print(bilist)
    for i in range(len(bilist)):
        if bilist[i] in bigramProb:

            outputProb1 *= bigramProb[bilist[i]]
        else:

            outputProb1 *= 0
    print('\n' + 'Probablility of sentence \"the weather\" = ' + str(outputProb1))

['the', 'weather', 'forecast', 'in', 'Koblenz', 'next', 'week', 'weather', 'forecast', 'the', 'weather', 'tomorrow', 'weather', 'forecast', 'Amsterdam', 'the', 'weather', 'tomorrow', 'in']

 All the possible Bigrams are 
[('the', 'weather'), ('weather', 'forecast'), ('forecast', 'in'), ('Koblenz', 'next'), ('next', 'week'), ('week', 'weather'), ('weather', 'forecast'), ('forecast', 'the'), ('the', 'weather'), ('weather', 'tomorrow'), ('tomorrow', 'weather'), ('weather', 'forecast'), ('Amsterdam', 'the'), ('the', 'weather'), ('weather', 'tomorrow'), ('tomorrow', 'in')]

 Bigrams along with their frequency 
{('the', 'weather'): 3, ('weather', 'forecast'): 3, ('forecast', 'in'): 1, ('Koblenz', 'next'): 1, ('next', 'week'): 1, ('week', 'weather'): 1, ('forecast', 'the'): 1, ('weather', 'tomorrow'): 2, ('tomorrow', 'weather'): 1, ('Amsterdam', 'the'): 1, ('tomorrow', 'in'): 1}

 Unigrams along with their frequency 
{'the': 3, 'weather': 5, 'forecast': 3, 'in': 1, 'Koblenz': 1, 'next': 1, 'w